In [236]:
# Import Libraries
import numpy as np
import pandas as pd
import geopandas as gp
import os

In [237]:
## Set working directory
os.chdir('E:/CR2/Repos/TNC-Demand-Model-Southeast/')

In [238]:
## Read in Data
trips = pd.read_csv('Cleaned_Inputs/Intermediate/neg_bin_pred_trips.csv')
trips.head()

,origin,destination,NT_TRIPS,AM_TRIPS,MD_TRIPS,PM_TRIPS,EV_TRIPS
0,21067000101,21067004007,0.000990,0.023043,0.087519,0.004800,0.003503
1,21067000101,21067003802,0.019028,0.237334,0.946659,0.147911,0.063125
2,21067000101,21067003918,0.014020,0.160490,0.895582,0.072979,0.039691
3,21067000101,21067004207,0.270208,1.453685,4.184008,0.940822,0.063206
4,21067000101,21067004006,0.000429,0.003241,0.033683,0.002471,0.001592


In [239]:
## Read in Data
traveltime = pd.read_csv('Cleaned_Inputs/Intermediate/pred_trips.csv')
traveltime = traveltime[['origin', 'destination', 'PRIVATE_TRAVEL_TIME', 'PRIVATE_TRIP_FARES', 'SHARED_TRAVEL_TIME', 'SHARED_TRIP_FARES']]
traveltime.head()

,origin,destination,PRIVATE_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRAVEL_TIME,SHARED_TRIP_FARES
0,21067003802,21067004007,40.016667,31.260439,44.016667,20.105323
1,21067004007,21067003802,36.616667,34.434858,40.616667,23.364816
2,21067004007,21067003701,40.533333,33.808273,44.533333,22.134277
3,21067003701,21067004007,38.750000,33.055025,42.750000,21.829425
4,21067004207,21067003918,32.583333,28.830300,36.583333,19.425726


In [240]:
## Read in Data
median_income = pd.read_csv('Raw_Inputs/Median Income/ACSDT5Y2019.B19013-Data.csv')
median_income.head()

,GEO_ID,NAME,B19013_001E,B19013_001M,Unnamed: 4
0,Geography,Geographic Area Name,Estimate!!Median household income in the past ...,Margin of Error!!Median household income in th...,NaN
1,0400000US21,Kentucky,50589,294,NaN
2,1400000US21001970100,"Census Tract 9701, Adair County, Kentucky",32071,9867,NaN
3,1400000US21001970200,"Census Tract 9702, Adair County, Kentucky",41741,9141,NaN
4,1400000US21001970300,"Census Tract 9703, Adair County, Kentucky",33413,3756,NaN


In [241]:
## Fix column names
median_income = median_income.rename(columns={'GEO_ID': 'tract', 'B19013_001E':'median_income'})
median_income.head()

,tract,NAME,median_income,B19013_001M,Unnamed: 4
0,Geography,Geographic Area Name,Estimate!!Median household income in the past ...,Margin of Error!!Median household income in th...,NaN
1,0400000US21,Kentucky,50589,294,NaN
2,1400000US21001970100,"Census Tract 9701, Adair County, Kentucky",32071,9867,NaN
3,1400000US21001970200,"Census Tract 9702, Adair County, Kentucky",41741,9141,NaN
4,1400000US21001970300,"Census Tract 9703, Adair County, Kentucky",33413,3756,NaN


In [242]:
## Clean Income Data
# Split out Name column and clear whitespace
median_income[['TRACT', 'COUNTY', 'STATE']] = median_income.NAME.str.split(',', expand=True)
median_income['COUNTY'] = median_income['COUNTY'].str.strip()
# Filter for Fayette County
median_income = median_income[median_income['COUNTY'] == 'Fayette County']
# Clean tract column by stipping string and converting it to a number
median_income['tract'] = median_income['tract'].str.replace('1400000US', '')
median_income.tract = median_income.tract.astype(np.int64)
# Keep relevant columns
median_income = median_income[['tract', 'median_income']]
## Convert income into $10,000 
median_income.median_income = median_income.median_income.astype(np.int64)
median_income['median_income'] = median_income['median_income']/10000
median_income.head()

,tract,median_income
264,21067000101,2.5363
265,21067000102,2.6477
266,21067000200,2.6725
267,21067000300,3.2089
268,21067000400,2.7031


In [243]:
median_income['tract'].nunique()

82

In [244]:
## Create origin and destination income tables
origin = median_income
origin = origin.rename(columns={"tract": "origin", "median_income": "origin_median_income"})
destination = median_income
destination = destination.rename(columns={"tract": "destination", "median_income": "destination_median_income"})

## Merge them with trips table
trips = pd.merge(trips, origin, how="left", on=["origin"])
trips = pd.merge(trips, destination, how="left", on=["destination"])
trips.head()

## Merge trips table with travel times
trips = pd.merge(trips, traveltime, how = "left", on = ["origin", "destination"])
trips.head()

## Add airport indicator
airport_census_tract = 21067004207
trips['airport'] = np.where(trips['origin'] == airport_census_tract, 1, 
                              np.where(trips['destination'] == airport_census_tract, 1, 0))

In [245]:
## Apply Table 5.7
## Use logit model to get probability
## Apply probabilities to trips
shared_exp_utility =  np.exp(-0.85 - 0.08*trips['SHARED_TRAVEL_TIME'] - 0.14*trips['SHARED_TRIP_FARES'] - 0.06*trips['origin_median_income'] - 0.06*trips['destination_median_income'] - 2.88*trips['airport'])
private_exp_utility = np.exp(-0.85 - 0.08*trips['PRIVATE_TRAVEL_TIME'] - 0.14*trips['PRIVATE_TRIP_FARES'])
trips['share_prob'] = shared_exp_utility / (shared_exp_utility + private_exp_utility)
trips['private_prob'] = 1 - trips[['share_prob']]
trips.head()

,origin,destination,NT_TRIPS,AM_TRIPS,MD_TRIPS,PM_TRIPS,EV_TRIPS,origin_median_income,destination_median_income,PRIVATE_TRAVEL_TIME,PRIVATE_TRIP_FARES,SHARED_TRAVEL_TIME,SHARED_TRIP_FARES,airport,share_prob,private_prob
0,21067000101,21067004007,0.000990,0.023043,0.087519,0.004800,0.003503,2.5363,8.1250,27.566667,20.880350,31.566667,13.710748,0,0.511015,0.488985
1,21067000101,21067003802,0.019028,0.237334,0.946659,0.147911,0.063125,2.5363,6.7212,19.533333,16.337407,23.533333,11.379667,0,0.454782,0.545218
2,21067000101,21067003918,0.014020,0.160490,0.895582,0.072979,0.039691,2.5363,8.7656,19.266667,14.882630,23.266667,10.216036,0,0.414646,0.585354
3,21067000101,21067004207,0.270208,1.453685,4.184008,0.940822,0.063206,2.5363,12.7798,23.933333,16.763696,27.933333,10.938764,1,0.035452,0.964548
4,21067000101,21067004006,0.000429,0.003241,0.033683,0.002471,0.001592,2.5363,22.8125,25.566667,17.017280,29.566667,10.854516,0,0.273261,0.726739


In [246]:
trips["share_prob"].describe()

count    6724.000000
mean        0.353283
std         0.088450
min         0.007603
25%         0.313646
50%         0.364560
75%         0.410938
max         0.589990
Name: share_prob, dtype: float64

In [247]:
trips["share_prob"][(trips['airport'] == 1)].describe()

count    163.000000
mean       0.029082
std        0.007259
min        0.007603
25%        0.023364
50%        0.030037
75%        0.034474
max        0.054081
Name: share_prob, dtype: float64

In [248]:
trips["share_prob"][(trips['airport'] == 0)].describe()

count    6561.000000
mean        0.361338
std         0.073075
min         0.039333
25%         0.318210
50%         0.367034
75%         0.412422
max         0.589990
Name: share_prob, dtype: float64

In [249]:
priv_share_trips = trips[['origin', 'destination', 'NT_TRIPS', 'AM_TRIPS', 'MD_TRIPS', 'PM_TRIPS', 'EV_TRIPS', 'share_prob', 'private_prob',"PRIVATE_TRAVEL_TIME", "SHARED_TRAVEL_TIME"]]
time_of_day = ["NT", "AM", "MD", "PM", "EV"]
mode_choices = ['private', 'share']
for time in time_of_day:
    for mode in mode_choices:
        priv_share_trips[mode + '_' + time + "_trips"] = priv_share_trips[time + '_TRIPS'] * priv_share_trips[mode + '_prob']
priv_share_trips.head()

C:\Users\Transportlab\AppData\Local\Temp\ipykernel_4756\1244184310.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  priv_share_trips[mode + '_' + time + "_trips"] = priv_share_trips[time + '_TRIPS'] * priv_share_trips[mode + '_prob']


,origin,destination,NT_TRIPS,AM_TRIPS,MD_TRIPS,PM_TRIPS,EV_TRIPS,share_prob,private_prob,PRIVATE_TRAVEL_TIME,...,private_NT_trips,share_NT_trips,private_AM_trips,share_AM_trips,private_MD_trips,share_MD_trips,private_PM_trips,share_PM_trips,private_EV_trips,share_EV_trips
0,21067000101,21067004007,0.000990,0.023043,0.087519,0.004800,0.003503,0.511015,0.488985,27.566667,...,0.000484,0.000506,0.011268,0.011775,0.042795,0.044723,0.002347,0.002453,0.001713,0.001790
1,21067000101,21067003802,0.019028,0.237334,0.946659,0.147911,0.063125,0.454782,0.545218,19.533333,...,0.010374,0.008654,0.129399,0.107936,0.516135,0.430524,0.080644,0.067267,0.034417,0.028708
2,21067000101,21067003918,0.014020,0.160490,0.895582,0.072979,0.039691,0.414646,0.585354,19.266667,...,0.008207,0.005813,0.093944,0.066547,0.524232,0.371350,0.042719,0.030261,0.023233,0.016458
3,21067000101,21067004207,0.270208,1.453685,4.184008,0.940822,0.063206,0.035452,0.964548,23.933333,...,0.260629,0.009579,1.402148,0.051536,4.035676,0.148332,0.907468,0.033354,0.060966,0.002241
4,21067000101,21067004006,0.000429,0.003241,0.033683,0.002471,0.001592,0.273261,0.726739,25.566667,...,0.000312,0.000117,0.002355,0.000886,0.024479,0.009204,0.001796,0.000675,0.001157,0.000435


In [250]:
priv_share_trips['shared_trips_total'] = priv_share_trips['share_NT_trips'] + priv_share_trips['share_NT_trips'] + priv_share_trips['share_AM_trips'] + priv_share_trips['share_MD_trips'] + priv_share_trips['share_PM_trips'] + priv_share_trips['share_EV_trips']
priv_share_trips.head()                                                 

,origin,destination,NT_TRIPS,AM_TRIPS,MD_TRIPS,PM_TRIPS,EV_TRIPS,share_prob,private_prob,PRIVATE_TRAVEL_TIME,...,share_NT_trips,private_AM_trips,share_AM_trips,private_MD_trips,share_MD_trips,private_PM_trips,share_PM_trips,private_EV_trips,share_EV_trips,shared_trips_total
0,21067000101,21067004007,0.000990,0.023043,0.087519,0.004800,0.003503,0.511015,0.488985,27.566667,...,0.000506,0.011268,0.011775,0.042795,0.044723,0.002347,0.002453,0.001713,0.001790,0.061754
1,21067000101,21067003802,0.019028,0.237334,0.946659,0.147911,0.063125,0.454782,0.545218,19.533333,...,0.008654,0.129399,0.107936,0.516135,0.430524,0.080644,0.067267,0.034417,0.028708,0.651742
2,21067000101,21067003918,0.014020,0.160490,0.895582,0.072979,0.039691,0.414646,0.585354,19.266667,...,0.005813,0.093944,0.066547,0.524232,0.371350,0.042719,0.030261,0.023233,0.016458,0.496242
3,21067000101,21067004207,0.270208,1.453685,4.184008,0.940822,0.063206,0.035452,0.964548,23.933333,...,0.009579,1.402148,0.051536,4.035676,0.148332,0.907468,0.033354,0.060966,0.002241,0.254622
4,21067000101,21067004006,0.000429,0.003241,0.033683,0.002471,0.001592,0.273261,0.726739,25.566667,...,0.000117,0.002355,0.000886,0.024479,0.009204,0.001796,0.000675,0.001157,0.000435,0.011435


In [251]:
np.sum(priv_share_trips['shared_trips_total'])

6632.207605099508

In [252]:
#### Read in Data for matching trips
households_load = pd.read_csv("Raw_Inputs/ACS/S1901/S1901.csv")
tracts_load = gp.read_file('Raw_Inputs/Shapefile Centroids/kentucky_census_centroids.shp')

In [253]:
#### Clean households data
households = households_load.drop([0, 1])
households = households.iloc[:, [0, 2]]
households = households.rename({'GEO_ID': 'tract', 'S1901_C01_001E': 'households'}, axis='columns')
households['tract'] = households['tract'].str.replace('1400000US', '')
households.tract = households.tract.astype(np.int64)
households.households = households.households.astype(np.int64)
households.head()

,tract,households
2,21067000101,1993
3,21067000102,984
4,21067000200,1225
5,21067000300,1249
6,21067000400,784


In [254]:
#### Clean tracts data
tracts = tracts_load[['GEOID', 'ALAND']]
tracts.columns = ['tract', 'land_area']
tracts.tract = tracts.tract.astype(np.int64)
### Convert from square meters to acres
tracts.loc[:, 'land_area'] = tracts.loc[:, 'land_area'] / 4046.85
tracts.head()

D:\Programs\anaconda3\envs\geo_env\lib\site-packages\pandas\core\generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
D:\Programs\anaconda3\envs\geo_env\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


,tract,land_area
0,21095971300,67653.550045
1,21211040302,1003.460469
2,21185030301,4974.180659
3,21107970600,1558.503775
4,21193971000,56921.420117


In [255]:
households = pd.merge(households, tracts, how="left", on=["tract"])
households['density'] = households['households'] / households['land_area']
households = households[["tract", "density"]]
households.head()

,tract,density
0,21067000101,6.192581
1,21067000102,5.049051
2,21067000200,3.152313
3,21067000300,4.667906
4,21067000400,2.675942


In [256]:
households['origin_hh_density'] = households['density']
households['dest_hh_density'] = households['density']
households = households.drop(["density"], axis = "columns")
origin_hh = households[["tract", "origin_hh_density"]]
dest_hh = households[["tract", "dest_hh_density"]]
households.head()

,tract,origin_hh_density,dest_hh_density
0,21067000101,6.192581,6.192581
1,21067000102,5.049051,5.049051
2,21067000200,3.152313,3.152313
3,21067000300,4.667906,4.667906
4,21067000400,2.675942,2.675942


In [257]:
matched_trips = pd.merge(priv_share_trips, origin_hh, left_on = ["origin"], right_on = ["tract"])
matched_trips = pd.merge(matched_trips, dest_hh, left_on = ["destination"], right_on = ["tract"])
matched_trips.head()

,origin,destination,NT_TRIPS,AM_TRIPS,MD_TRIPS,PM_TRIPS,EV_TRIPS,share_prob,private_prob,PRIVATE_TRAVEL_TIME,...,share_MD_trips,private_PM_trips,share_PM_trips,private_EV_trips,share_EV_trips,shared_trips_total,tract_x,origin_hh_density,tract_y,dest_hh_density
0,21067000101,21067004007,0.000990,0.023043,0.087519,0.004800,0.003503,0.511015,0.488985,27.566667,...,0.044723,0.002347,0.002453,0.001713,0.001790,0.061754,21067000101,6.192581,21067004007,0.027091
1,21067000102,21067004007,0.000626,0.015568,0.053756,0.002770,0.002261,0.516114,0.483886,28.216667,...,0.027744,0.001340,0.001429,0.001094,0.001167,0.039021,21067000102,5.049051,21067004007,0.027091
2,21067000200,21067004007,0.000276,0.010466,0.031179,0.001409,0.001071,0.538416,0.461584,30.566667,...,0.016787,0.000650,0.000759,0.000494,0.000577,0.024055,21067000200,3.152313,21067004007,0.027091
3,21067000300,21067004007,0.000278,0.011647,0.030928,0.001310,0.001042,0.510117,0.489883,28.450000,...,0.015777,0.000642,0.000668,0.000511,0.000532,0.023202,21067000300,4.667906,21067004007,0.027091
4,21067000400,21067004007,0.000306,0.010772,0.033860,0.001669,0.001232,0.507273,0.492727,27.383333,...,0.017176,0.000822,0.000846,0.000607,0.000625,0.024423,21067000400,2.675942,21067004007,0.027091


In [261]:
#### Utility Equation for Matched Trips
match_exp_u = np.exp(-1.45 + 0.148*matched_trips["SHARED_TRAVEL_TIME"] + 0.323*matched_trips["shared_trips_total"] + 0.012*matched_trips['origin_hh_density'] + 0.016*matched_trips["dest_hh_density"])
matched_trips["matched_prob"] = match_exp_u / (1 + match_exp_u)
matched_trips.head(10)

,origin,destination,NT_TRIPS,AM_TRIPS,MD_TRIPS,PM_TRIPS,EV_TRIPS,share_prob,private_prob,PRIVATE_TRAVEL_TIME,...,private_PM_trips,share_PM_trips,private_EV_trips,share_EV_trips,shared_trips_total,tract_x,origin_hh_density,tract_y,dest_hh_density,matched_prob
0,21067000101,21067004007,0.000990,0.023043,0.087519,0.004800,0.003503,0.511015,0.488985,27.566667,...,0.002347,0.002453,0.001713,0.001790,0.061754,21067000101,6.192581,21067004007,0.027091,0.964992
1,21067000102,21067004007,0.000626,0.015568,0.053756,0.002770,0.002261,0.516114,0.483886,28.216667,...,0.001340,0.001429,0.001094,0.001167,0.039021,21067000102,5.049051,21067004007,0.027091,0.967444
2,21067000200,21067004007,0.000276,0.010466,0.031179,0.001409,0.001071,0.538416,0.461584,30.566667,...,0.000650,0.000759,0.000494,0.000577,0.024055,21067000200,3.152313,21067004007,0.027091,0.976152
3,21067000300,21067004007,0.000278,0.011647,0.030928,0.001310,0.001042,0.510117,0.489883,28.450000,...,0.000642,0.000668,0.000511,0.000532,0.023202,21067000300,4.667906,21067004007,0.027091,0.968218
4,21067000400,21067004007,0.000306,0.010772,0.033860,0.001669,0.001232,0.507273,0.492727,27.383333,...,0.000822,0.000846,0.000607,0.000625,0.024423,21067000400,2.675942,21067004007,0.027091,0.962136
5,21067000500,21067004007,0.000891,0.019776,0.091341,0.005711,0.003560,0.410284,0.589716,23.600000,...,0.003368,0.002343,0.002100,0.001461,0.050124,21067000500,4.334342,21067004007,0.027091,0.937229
6,21067000600,21067004007,0.000980,0.020337,0.071745,0.003993,0.003314,0.361023,0.638977,23.383333,...,0.002551,0.001441,0.002118,0.001197,0.036589,21067000600,4.116599,21067004007,0.027091,0.934892
7,21067000700,21067004007,0.000943,0.019067,0.071617,0.003884,0.003101,0.482427,0.517573,25.366667,...,0.002010,0.001874,0.001605,0.001496,0.048028,21067000700,5.319002,21067004007,0.027091,0.951480
8,21067000801,21067004007,0.001033,0.018368,0.089210,0.005281,0.003647,0.496895,0.503105,25.883333,...,0.002657,0.002624,0.001835,0.001812,0.058918,21067000801,0.066403,21067004007,0.027091,0.952257
9,21067000802,21067004007,0.000504,0.012989,0.043434,0.002201,0.001812,0.510821,0.489179,27.233333,...,0.001077,0.001124,0.000886,0.000925,0.031387,21067000802,3.155634,21067004007,0.027091,0.961616


In [262]:
matched_trips["matched_prob"].describe()

count    6724.000000
mean        0.834829
std         0.106343
min         0.300198
25%         0.782758
50%         0.858112
75%         0.912168
max         0.999836
Name: matched_prob, dtype: float64

In [263]:
matched_trips["unmatched_prob"] = 1 - matched_trips["matched_prob"]

In [265]:
time_of_day = ["NT", "AM", "MD", "PM", "EV"]
for time in time_of_day:
    matched_trips["matched_" + time + "_trips"] = matched_trips["share_" + time + "_trips"] * matched_trips["matched_prob"]
    matched_trips["unmatched_" + time + "_trips"] = matched_trips["share_" + time + "_trips"] * matched_trips["unmatched_prob"]
matched_trips.head(10)

,origin,destination,NT_TRIPS,AM_TRIPS,MD_TRIPS,PM_TRIPS,EV_TRIPS,share_prob,private_prob,PRIVATE_TRAVEL_TIME,...,matched_NT_trips,unmatched_NT_trips,matched_AM_trips,unmatched_AM_trips,matched_MD_trips,unmatched_MD_trips,matched_PM_trips,unmatched_PM_trips,matched_EV_trips,unmatched_EV_trips
0,21067000101,21067004007,0.000990,0.023043,0.087519,0.004800,0.003503,0.511015,0.488985,27.566667,...,0.000488,0.000018,0.011363,0.000412,0.043158,0.001566,0.002367,0.000086,0.001727,0.000063
1,21067000102,21067004007,0.000626,0.015568,0.053756,0.002770,0.002261,0.516114,0.483886,28.216667,...,0.000312,0.000011,0.007773,0.000262,0.026841,0.000903,0.001383,0.000047,0.001129,0.000038
2,21067000200,21067004007,0.000276,0.010466,0.031179,0.001409,0.001071,0.538416,0.461584,30.566667,...,0.000145,0.000004,0.005501,0.000134,0.016387,0.000400,0.000741,0.000018,0.000563,0.000014
3,21067000300,21067004007,0.000278,0.011647,0.030928,0.001310,0.001042,0.510117,0.489883,28.450000,...,0.000138,0.000005,0.005752,0.000189,0.015275,0.000501,0.000647,0.000021,0.000515,0.000017
4,21067000400,21067004007,0.000306,0.010772,0.033860,0.001669,0.001232,0.507273,0.492727,27.383333,...,0.000149,0.000006,0.005258,0.000207,0.016526,0.000650,0.000814,0.000032,0.000601,0.000024
5,21067000500,21067004007,0.000891,0.019776,0.091341,0.005711,0.003560,0.410284,0.589716,23.600000,...,0.000342,0.000023,0.007604,0.000509,0.035123,0.002352,0.002196,0.000147,0.001369,0.000092
6,21067000600,21067004007,0.000980,0.020337,0.071745,0.003993,0.003314,0.361023,0.638977,23.383333,...,0.000331,0.000023,0.006864,0.000478,0.024215,0.001686,0.001348,0.000094,0.001119,0.000078
7,21067000700,21067004007,0.000943,0.019067,0.071617,0.003884,0.003101,0.482427,0.517573,25.366667,...,0.000433,0.000022,0.008752,0.000446,0.032874,0.001676,0.001783,0.000091,0.001423,0.000073
8,21067000801,21067004007,0.001033,0.018368,0.089210,0.005281,0.003647,0.496895,0.503105,25.883333,...,0.000489,0.000025,0.008691,0.000436,0.042212,0.002116,0.002499,0.000125,0.001726,0.000087
9,21067000802,21067004007,0.000504,0.012989,0.043434,0.002201,0.001812,0.510821,0.489179,27.233333,...,0.000248,0.000010,0.006380,0.000255,0.021336,0.000852,0.001081,0.000043,0.000890,0.000036


In [266]:
matched_trips.to_csv("outputs/fayette_county_ky_matched_trips.csv", index = False)